# Start Up

Run the cells below. They will open a browser window through selenium. (check your open windows if you don't notice it. It will be a firefox page with an orange url bar.)

Log into your ESPN account and navigate to the draft.

Make sure to keep everything in a single tab. Do not open links as new windows, do not open links as new tabs. This should be the default behavior if you just click on things normally (but no promises - this is hacked together at best...)

### Updating the probabilities

The default score/pick probability info is for 10 team, standard (non-ppr) scoring.

You can update the probabilities with different parameters ```grandmaster.create_prob_pickle(teams, outputname, mapping_path, scoring, outputpath)``` where teams is the number of teams in your league, and scoring is one of these four entries:
 - 'standard'
 - 'ppr'
 - 'half-ppr'
 - '2qb

The other parameters deal with file mappings.
 
Follow this link to see the source data [Fantasy Football Calculator](https://fantasyfootballcalculator.com/scenario-calculator)

### Updating/Changing the Projections

You can call ```grandmaster.process_projections(projection_path, idmap_path, output_path)```
to update or change the projections. ```projection_path``` is the direction to the projection files. I use [fantasyfootballanalytics.net](https://www.fantasyfootballanalytics.net)

You can use any source of projections as long as you can map them to the ESPN IDs. 

Required columns:
 - 'position'
 - 'player'
 - 'points'
 - 'adp'
 
This needs more development, specifically:
 - mapping tables to cross-reference all the different sources to some sort of master ID
 - It would also be useful to map the player IDs to the different providers.
 - making the column naming more flexible 
 
 
 # Current Use.
 I've put together a rough GUI using widgets. Follow the instructions [here](https://ipywidgets.readthedocs.io/en/latest/user_install.html) to install and enable them.
 
 Alternatively, there is another notebook in the git repo labeled No GUI or something that just calls the functions directly.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import pandas as pd 
import numpy as np
import grandmaster as gm

In [ ]:
process_proj = interactive(gm.process_projections,
                           {'manual':True},
                           projection_path='https://raw.githubusercontent.com/jwaggie14/footb/master/ffa_customrankings2020-0.csv',
                           idmap_path='https://raw.githubusercontent.com/jwaggie14/footb/master/espn_ffa_map.csv',
                           output_path='players_NEW.pkl'
                          )

update_prob = interactive(gm.create_prob_pickle,
                          {'manual':True},
                          teams=[8,10,12,14],
                          scoring=['standard','ppr','half-ppr','2qb'],
                          outputname='pick_prob_NEW.pkl',
                          mapping_path='https://raw.githubusercontent.com/jwaggie14/footb/master/probmap.csv'
                         )     

# Update Projections
fill in the fields and click "Run Interact" to process projections.

The projection_path defaults to the projections that I've stored in the github repository. These are the projections for standard, non-ppr scoring.

You can go to [fantasyfootballanalytics.net](https://www.fantasyfootballanalytics.net) to create projections that match your league's settings. Download the custom rankings. Update the projection_path to point to your custom rankings file.

In [ ]:
display(process_proj)

# Update Pick Probabilities
select the closest Team number and Scoring settings, fill in the mapping path and outputname,
then click "Run Interact". Scraping this info will take a minute. 

Only scrape the picks once a day. The probabilities don't change very much, so there is no need to swamp their website with requests.

In [ ]:
display(update_prob)

In [ ]:
def start_draft(team_name,proj_path,prob_path):
    global projections
    global probabilities
    global d
    print('1 of 4: reading projections')
    projections = pd.read_pickle(proj_path)
    print('2 of 4: reading pick probabilities')
    probabilities = pd.read_pickle(prob_path)
    print('3 of 4: launching driver - this will take a minute')
    d = gm.ds.draft_monitor(team_name)
    print('4 of 4: done')

In [ ]:
x = interactive(start_draft,
                {'manual':True},
                team_name='Team LASTNAME',
                proj_path=fixed(process_proj.children[2].value),
                prob_path=fixed(update_prob.children[2].value)
               )

# Launch your draft
Fill in your espn team name and press Run Interact again. This will bring up a new firefox window that you will use for your actual draft.

Use this new window to log into your espn account and navigate to your draft room.

In [ ]:
display(x)

If you made a typo in your team name, you will get a "'Team Whatever' is not in list" error.
Change your team name with ```d.team_name = "correct team name here"```

In [ ]:
d.configure_draft()

Run ```d.myteam``` to make sure the number matches your first round pick.
If it's wrong, change it with ```d.myteam = YOURPICK```

There are also some other info that you can dig into if you're interested like rosters.

In [ ]:
d.myteam

In [ ]:
d.rosters[d.myteam]

If you just want recommendations, run ```gm.tell_me_what_to_do()``` on your every turn.

In [ ]:
oc = gm.tell_me_what_to_do(projections,probabilities,d)
cols = ['player','team','position','points','adp','oc_raw_0','oc_adj_0','oc_adj_1']
print('Roster Needs')
print(d.empty_positions)
oc[cols].sort_values('oc_adj_1',ascending=False).head(10)

# Remove players you don't want

Leonard Fournette was just released from the Jaguars - so we know his projection data doesn't represent his current prospects. 

We can add him to the blacklist so that he doesn't pop up as a recommendation.

In [ ]:
projections = gm.dl.blacklist_player('Leonard Fournette','JAC',projections)

You can also remove him from the blacklist if you change your mind, or added by mistake or whatever. Just use the same syntax, but its ```unblacklist_player()``` instead of ```blacklist_player()```. Creative I know...

In [ ]:
projections = gm.dl.unblacklist_player('Leonard Fournette','JAC',projections)

Feel free to explore the package and classes while you wait for your turn to return